## Introduction

In this tutorial, you will learn how to:
1. Use the Fiedler vector to identify **bridge edges** (bottlenecks) in a network
2. Compute **edge energy** to quantify the tension across each edge
3. Implement an algorithm to find **communities** (groups of nodes tightly connected) by iteratively removing bridges.

**Key concept**: The Fiedler vector $\mathbf{w}_2$ minimizes edge energy $\sum_{(i,j)} A_{ij}(w_2(i) - w_2(j))^2$. Edges with large $(w_2(i) - w_2(j))^2$ are **bridges** that weakly connect otherwise dense regions.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

### Helper functions (PROVIDED)

In [ ]:
def ring_of_cliques_adjacency(clique_size: int = 5, n_cliques: int = 4):
    """Return cliques connected in a ring by single bridging edges."""
    cliques = [nx.complete_graph(clique_size) for _ in range(n_cliques)]
    ring = nx.disjoint_union_all(cliques)
    offsets = [i * clique_size for i in range(n_cliques)]
    for i in range(n_cliques):
        ring.add_edge(offsets[i], offsets[(i + 1) % n_cliques])
    return nx.to_numpy_array(ring, dtype=float)


def plot_network_with_edge_colors(
    adj_matrix,
    node_colors = None,
    edge_energies = None,
    title: str = "",
) -> None:
    """Visualize network with nodes colored by Fiedler vector and edges by energy."""
    G = nx.from_numpy_array(adj_matrix)
    pos = nx.spring_layout(G, seed=42)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Draw edges
    if edge_energies is not None:
        edges = list(edge_energies.keys())
        weights = [edge_energies[e] for e in edges]
        # Normalize for visualization
        if len(weights) > 0:
            max_weight = max(weights)
            widths = [1 + 5 * (w / max_weight) for w in weights]
        else:
            widths = [1] * len(edges)
        nx.draw_networkx_edges(G, pos, edgelist=edges, width=widths, 
                             edge_color=weights, edge_cmap=plt.cm.Reds,
                             edge_vmin=0, ax=ax)
    else:
        nx.draw_networkx_edges(G, pos, ax=ax)
    
    # Draw nodes
    if node_colors is not None:
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                              cmap=plt.cm.coolwarm, node_size=200, ax=ax)
    else:
        nx.draw_networkx_nodes(G, pos, node_size=200, ax=ax)
    
    ax.set_title(title)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

## Section 1: Computing Edge Energy

### Task 1: Implement edge energy computation

The **edge energy** for edge $(i,j)$ with respect to a vector $\mathbf{v}$ is:
$$
E_{ij}[\mathbf{v}] = A_{ij} \cdot (v_i - v_j)^2
$$

For the Fiedler vector $\mathbf{w}_2$, edges with high energy are bottlenecks.

**Your task**:
1. Implement `fiedler_vector(adj_matrix)` that computes and returns the Fiedler vector of the graph represented by `adj_matrix`.
2. Implement `compute_edge_energy(adj_matrix, vector)` that returns a dictionary mapping each edge `(i, j)` to its energy.

In [ ]:
def fiedler_vector(adj_matrix: np.ndarray):
    """Return the algebraic connectivity and its corresponding eigenvector."""
   
    return lambda_two, fiedler_vec

In [ ]:
def compute_edge_energy(adj_matrix: np.ndarray, vector: np.ndarray):
    """
    Returns:
        Dict: Mapping from edge (i, j) to its energy.
    """
    
    return edge_energy

### Test your implementation running the following cells:

In [ ]:
# Create a simple ring of 6 cliques of 5 nodes each
adj = ring_of_cliques_adjacency(clique_size=5, n_cliques=6)

# Compute Fiedler vector
omega2, w2 = fiedler_vector(adj)
print(f"Algebraic connectivity: {omega2:.4f}")

# Compute edge energies
edge_energy = compute_edge_energy(adj, w2)

# Display top 5 edges with highest energy (these should be bridges)
sorted_edges = sorted(edge_energy.items(), key=lambda x: x[1], reverse=True)
print("\nTop 5 edges with highest energy:")
for edge, energy in sorted_edges[:5]:
    print(f"  Edge {edge}: energy = {energy:.4f}")

In [ ]:
# Visualize the network
plot_network_with_edge_colors(adj, node_colors=w2, edge_energies=edge_energy, title="Ring of cliques: edges colored by energy")

## Section 2: Identifying Bridge Edges

### Task 2.1: Implement bridge detection

**Your task**:
1. Implement `find_bridge_edges(adj_matrix, n_bridges)` that:
   - Computes the Fiedler vector
   - Computes edge energies
   - Returns the `n_bridges` edges with highest energy as a list of tuples.
2. Test on the ring-of-cliques graph

In [ ]:
def find_bridge_edges(adj_matrix, n_bridges = 1):
    
    return bridges

In [ ]:
# Test on the adjacency of the graph created above
bridges = find_bridge_edges(adj, n_bridges=4)

print("Identified bridge edges:")
for edge in bridges:
    print(f"  {edge}")

## Section 3: Block Decomposition by Edge Removal (Challenging task)

### Task 3: Implement iterative block decomposition

**The Challenge**: We want to identify natural communities (blocks) by removing bridge edges. But how do we decide which edges to remove?

**Naive approach (doesn't work)**: Simply set an energy threshold and remove all edges above it. Problem: as we remove edges, the network gets smaller, and energy scales change dramatically. A "low" energy in a large component becomes "high" in a small one.

**Solution: Z-score normalization**

The **z-score** (or standard score) measures how many standard deviations a value is from the mean:
$$
z = \frac{x - \mu}{\sigma}
$$
where $\mu$ is the mean and $\sigma$ is the standard deviation.

**The Algorithm**:

**Input**: Adjacency matrix, z-score threshold $\theta$, minimum component size

**Repeat until all components are "blocked"**:
1. Find all connected components in current graph
2. For each component:
   - Compute Fiedler vector $\mathbf{w}_2$ and edge energies $E_{ij} = (w_2(i) - w_2(j))^2$
   - Compute mean energy: $\mu = \frac{1}{m}\sum_{(i,j)} E_{ij}$
   - Compute standard deviation: $\sigma = \sqrt{\frac{1}{m}\sum_{(i,j)} (E_{ij} - \mu)^2}$
   - Compute z-score for each edge: $z_{ij} = \frac{E_{ij} - \mu}{\sigma}$
3. Select the component with **lowest algebraic connectivity** $\omega_2$ (the one where diffusion is slowest)
4. In that component, find edge with **highest z-score** (the outlier)
5. **If** z-score $> \theta$: 
   - Remove the edge
   - Continue to next iteration
6. **Else**: 
   - Mark component as "blocked" (no more bridges are present)
   - Continue with other components

**Output**: List of blocks
Example of output:
list of blocks: `[[0,1,2],[3,4,5],[6,7]]` --> three communities in three lists. Each list contains node indices belonging to that community.

#### SUGGESTIONS:
To find connected components, you can use NetworkX:
```python
G = nx.from_numpy_array(adj) # create graph from adjacency matrix
components = list(nx.connected_components(G)) # get connected components
```

In [ ]:
def decompose_into_blocks(adj_matrix, energy_threshold = 2.0):
    
    adj = adj_matrix.copy() # Create a copy since you may modify it in-place within the function

    # YOUR CODE HERE

    return blocks

### Test on ring of cliques by running this code

In [ ]:
# Decompose into blocks (z-score threshold of 2.0 means edge must be 2 std above mean)
blocks = decompose_into_blocks(adj, energy_threshold=3.5)

print(f"Found {len(blocks)} blocks:")
for i, block in enumerate(blocks):
    print(f"  Block {i+1}: {len(block)} nodes")

# Visualize the decomposition with the code provided below

In [ ]:
def plot_blocks(adj_matrix, blocks):
    """Visualize network with nodes colored by block membership."""
    G = nx.from_numpy_array(adj_matrix)
    pos = nx.spring_layout(G, seed=42)
    
    # Assign color to each node based on block
    n = adj_matrix.shape[0]
    node_colors = np.zeros(n)
    for block_id, block in enumerate(blocks):
        for node in block:
            node_colors[node] = block_id
    
    fig, ax = plt.subplots(figsize=(8, 6))
    nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=1, ax=ax)
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          cmap=plt.cm.tab10, node_size=200, ax=ax)
    ax.set_title(f"Network decomposed into {len(blocks)} blocks")
    ax.axis('off')
    plt.tight_layout()
    plt.show()

plot_blocks(adj, blocks)